# Abstractive summarization with T5 Transformer model

## Introduction

### Automatic Summarization

__Automatic summarization__ is one of the central problems in Natural Language Processing (NLP). Summarization consists on creating a shorter version of a document or an article that captures all the important information. It poses several challenges relating to language understanding (e.g. identifying important content) and generation (e.g. aggregating and rewording the identified content into a summary).
Along with translation, it is another example of a task that can be formulated as a sequence-to-sequence task. 

Summarization can be: 

- _Extractive_: extract the most relevant information from a document.
- _Abstractive_: generate new text that captures the most relevant information.

In this project we will approach the problem of single-document abstractive summarization. Following prior work, we aim to tackle this problem using a sequence-to-sequence model. 

### The T5 model

[Text-to-Text Transfer Transformer (T5)](https://arxiv.org/abs/1910.10683) is a [Transformer-based](https://arxiv.org/abs/1706.03762) model built on the encoder-decoder architecture, pretrained on a multi-task mixture of unsupervised and supervised tasks where each task is converted into a text-to-text format. T5 shows impressive results in a variety of sequence-to-sequence tasks like summarization, translation, etc.

The T5 model was presented in [Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer](https://arxiv.org/pdf/1910.10683) by Colin Raffel, Noam Shazeer, Adam Roberts, Katherine Lee, Sharan Narang, Michael Matena, Yanqi Zhou, Wei Li, Peter J. Liu.

Some notes on the model:

1. T5 is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks and for which each task is converted into a text-to-text format. T5 works well on a variety of tasks out-of-the-box by prepending a different prefix to the input corresponding to each task, e.g., for translation: translate English to German: …, for summarization: summarize: ….

1. The pretraining includes both supervised and self-supervised training. Supervised training is conducted on downstream tasks provided by the GLUE and SuperGLUE benchmarks (converting them into text-to-text tasks as explained above).

1. Self-supervised training uses corrupted tokens, by randomly removing 15% of the tokens and replacing them with individual sentinel tokens (if several consecutive tokens are marked for removal, the whole group is replaced with a single sentinel token). The input of the encoder is the corrupted sentence, the input of the decoder is the original sentence and the target is then the dropped out tokens delimited by their sentinel tokens.

1. T5 uses relative scalar embeddings. Encoder input padding can be done on the left and on the right.

Alternative models that can be used in this notebook instead of T5 include:

[BART](https://huggingface.co/docs/transformers/model_doc/bart), [BigBird-Pegasus](https://huggingface.co/docs/transformers/model_doc/bigbird_pegasus), [Blenderbot](https://huggingface.co/docs/transformers/model_doc/blenderbot), [BlenderbotSmall](https://huggingface.co/docs/transformers/model_doc/blenderbot-small), [Encoder decoder](https://huggingface.co/docs/transformers/model_doc/encoder-decoder), [FairSeq Machine-Translation](https://huggingface.co/docs/transformers/model_doc/fsmt), [LED](https://huggingface.co/docs/transformers/model_doc/led), [LongT5](https://huggingface.co/docs/transformers/model_doc/longt5), [M2M100](https://huggingface.co/docs/transformers/model_doc/m2m_100), [Marian](https://huggingface.co/docs/transformers/model_doc/marian), [mBART](https://huggingface.co/docs/transformers/model_doc/mbart), [MT5](https://huggingface.co/docs/transformers/model_doc/mt5), [MVP](https://huggingface.co/docs/transformers/model_doc/mvp), [NLLB](https://huggingface.co/docs/transformers/model_doc/nllb), [NLLB-MOE](https://huggingface.co/docs/transformers/model_doc/nllb-moe), [Pegasus](https://huggingface.co/docs/transformers/model_doc/pegasus), [PEGASUS-X](https://huggingface.co/docs/transformers/model_doc/pegasus_x), [PLBart](https://huggingface.co/docs/transformers/model_doc/plbart), [ProphetNet](https://huggingface.co/docs/transformers/model_doc/prophetnet), [SwitchTransformers](https://huggingface.co/docs/transformers/model_doc/switch_transformers), [UMT5](https://huggingface.co/docs/transformers/model_doc/umt5), [XLM-ProphetNet](https://huggingface.co/docs/transformers/model_doc/xlm-prophetnet)

### The XSum Dataset

In this notebook, we will fine-tune the pretrained T5 model on the Abstractive Summarization task using Hugging Face Transformers on the [Extreme Summarization (XSum)](https://arxiv.org/abs/1808.08745) dataset loaded from Hugging Face Datasets. We will then use this finetuned model for inference.

## Setup
### Installing the requirements

In [1]:
%pip install -Uqq numpy scipy transformers datasets evaluate rouge_score accelerate

Note: you may need to restart the kernel to use updated packages.


### Define variables

In [6]:
DATASET_SAMPLE_SPLIT = 0.2 # The percentage of the dataset we will sample
TRAIN_SIZE_SPLIT = 0.8 # The percentage of the sample dataset that we will use in the train set
TEST_SIZE_SPLIT = 0.2 # The percentage of the sample dataset that we will use in the test set
MAX_INPUT_LENGTH = 1024  # Maximum length of the input to the model
MIN_TARGET_LENGTH = 5  # Minimum length of the output by the model
MAX_TARGET_LENGTH = 128  # Maximum length of the output by the model
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# This notebook is built on the t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "t5-small"

# Local directory where to save the finetuned model
MODEL_PATH = "T5-XSum-base"

# Repository name for saving model to the Hugging Face Hub
REPO_NAME = "alexrodpas/T5-XSum-base"

# File for inference example
INPUT_FILE = "Input/Airlines_Are_Just_Banks_Now.txt"

# For summarization tasks, T5 requires the following prefix
PREFIX = "summarize: "

# Enable parallelized tokenization
TOKENIZERS_PARALLELISM = True

# Disable W&B logging
import os
os.environ["WANDB_DISABLED"] = "true"

## Load the dataset
We will now download the [Extreme Summarization (XSum)](https://arxiv.org/abs/1808.08745) dataset. This dataset consists of BBC articles and accompanying single sentence summaries. Specifically, each article is prefaced with an introductory sentence (aka summary) which is professionally written, typically by the author of the article. That dataset has 226,711 articles divided into training (90%, 204,045), validation (5%, 11,332), and test (5%, 11,334) sets.

We will use the Recall-Oriented Understudy for Gisting Evaluation (ROUGE) metric, which is commonly found in the literature, to evaluate our sequence-to-sequence abstrative summarization approach.

We will use the [Hugging Face Datasets library](https://github.com/huggingface/datasets) to download the data we need to use for training and evaluation. This can be easily done with the `load_dataset` function.

In [7]:
from datasets import load_dataset

raw_datasets = load_dataset("xsum", split="train")

The dataset has the following fields:

- __document__: the original BBC article to be summarized
- __summary__: the single sentence summary of the BBC article
- __id__: ID of the document-summary pair

In [8]:
raw_datasets

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})

We can see how the data looks like by retrieving the first item in ``raw_datasets``:

In [9]:
print(raw_datasets[0])

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

For the sake of demonstrating the workflow, in this notebook we will only take small stratified balanced splits (20%) of the dataset as our training and test sets (which are then split 80/20). We can easily split the dataset using the `train_test_split` method which expects the split size and the name of the column relative to which you want to stratify.

In [10]:
raw_datasets = raw_datasets.train_test_split(train_size=DATASET_SAMPLE_SPLIT*TRAIN_SIZE_SPLIT, test_size=DATASET_SAMPLE_SPLIT*TEST_SIZE_SPLIT)

In [11]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 32647
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 8162
    })
})

## Data pre-processing
Before we can feed those texts to our model, we need to pre-process them and get them ready for the task. This is done by a Hugging Face Transformers `Tokenizer` which will tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

The `from_pretrained()` method expects the name of a model from the Hugging Face Model Hub. This is the MODEL_CHECKPOINT that we declared earlier.

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Next, we will write a simple function that helps us in the pre-processing that is compatible with Hugging Face Datasets. This pre-processing function should:

1. Properly indicate the task that we intend to perform, which is summarization (T5 models can also be used for translation, so if using one of the five T5 checkpoints we have to prefix the inputs with "summarize:").
1. Tokenize the text dataset (input and targets) into it's corresponding token ids that will be used for embedding look-up in BERT.
1. Add the prefix to the tokens.
1. Create additional inputs for the model like `token_type_ids`, `attention_mask`, etc.


In [13]:
def preprocess_fn(examples, tokenizer):
    if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]: prefix = PREFIX
    else: prefix = ""

    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(text=examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training and testing data will be preprocessed in one single command.

In [14]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 32647
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 8162
    })
})

In [15]:
tokenized_datasets = raw_datasets.map(preprocess_fn, fn_kwargs={"tokenizer":tokenizer}, batched=True)

Map:   0%|          | 0/32647 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/8162 [00:00<?, ? examples/s]

## Define the model
Now we can download the pretrained model and fine-tune it. Since our task is sequence-to-sequence (both the input and output are text sequences), we use the `TFAutoModelForSeq2SeqLM` class from the Hugging Face Transformers library. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

The `from_pretrained()` method expects the name of a model from the Hugging Face Model Hub. As mentioned earlier, we will use the `t5-small` model checkpoint.

In [16]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


For training Sequence to Sequence models, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels. Thus, we use the `DataCollatorForSeq2Seq` provided by the Hugging Face Transformers library on our dataset.

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## Define evaluation metric and function
We define a metric that we'll use during training because it will help in evaluating our model’s performance. We can quickly load a evaluation method with the `evaluate` library. For our summarization task, we load the [ROUGE](https://huggingface.co/spaces/evaluate-metric/rouge) metric.

In [18]:
import evaluate

rouge = evaluate.load("rouge")

Now we define `metric_fn` which will calculate the ROUGE score between the ground-truth and predictions.

In [19]:
import numpy as np

def metric_fn(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Train the model

We will use [``Seq2SeqTrainer``](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer), a ``Trainer`` class from the Transformers library that can be used for sequence-to-sequence tasks such as translation or summarization.

To train the model we:

1. Define our training hyperparameters in `Seq2SeqTrainingArguments`. The only required parameter is `output_dir` which specifies where to save our model. At the end of each epoch, the `Trainer` will evaluate the ROUGE metric and save the training checkpoint. We set `push_to_hub` to True so we can later upload the model to the Hugging Face hub (this requires being logged in to Hugging Face).
2. Pass the training arguments to `Seq2SeqTrainer` along with the model, dataset, tokenizer, data collator, and compute_metrics function.
3. Call `train()` to finetune our model.

In [21]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_PATH,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=metric_fn,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.823400,2.591606,0.262300,0.064700,0.204300,0.204400,18.815200
2,2.774200,2.557712,0.270700,0.070200,0.211800,0.211700,18.821200
3,2.748200,2.549059,0.273000,0.071100,0.213400,0.213400,18.819400


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=6123, training_loss=2.8180970773380722, metrics={'train_runtime': 3950.0294, 'train_samples_per_second': 24.795, 'train_steps_per_second': 1.55, 'total_flos': 2.5937796565794816e+16, 'train_loss': 2.8180970773380722, 'epoch': 3.0})

## Save the model

First we save the model, locally, for future use:

In [23]:
trainer.save_model(MODEL_PATH)

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

events.out.tfevents.1699377594.n22hf832wh.81.0:   0%|          | 0.00/8.94k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

Now, we can also upload it (push it) to the Hugging Face Hub, so we can share it and get version control on it. Since Hugging Face model repos are just Git repositories, we could use Git to push our model files to the Hub. Follow the guide on [Getting Started with Repositories](https://huggingface.co/docs/hub/repositories-getting-started) to learn about using the git CLI to commit and push your models. Here we will use the `huggingface_hub` client library to upload our model to the Hub. The rich feature set in the `huggingface_hub` library allows you to manage repositories, including creating repos and uploading models to the Model Hub. Visit the client library’s documentation to learn more. We'll need a Hugging Face API token (either stored in the cache or copied and pasted in our notebook). You can find in the [Hugging Face documentation](https://huggingface.co/docs/transformers/v4.14.1/model_sharing) how to obtain it.

In [ ]:
from huggingface_hub import login

login()

In [25]:
trainer.push_to_hub(REPO_NAME)

'https://huggingface.co/alexrodpas/T5-XSum-base/tree/main/'

## Inference

Once our model is finetuned, we can use it for inference. We'll show here an example, with the following text (extracted from the [Airlines Are Just Banks Now](https://www.theatlantic.com/ideas/archive/2023/09/airlines-banks-mileage-programs/675374/) article by Ganesh Sitaraman, published in The Atlantic on September 21, as input.

In [27]:
input_file = INPUT_FILE
with open(input_file, 'r') as file:
    input = file.read().replace('\n', '')

We add the prefix required by T5 for summarization:

In [28]:
pref_input = PREFIX + input

The simplest way to try out our finetuned model for inference is to use it in a ``pipeline()``. We instantiate a pipeline for summarization with our model, and pass our input text to it:

In [29]:
from transformers import pipeline

summarizer = pipeline("summarization", model=REPO_NAME)
summarizer(pref_input)

Token indices sequence length is longer than the specified maximum sequence length for this model (2097 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'American Airlines has made three changes to its mileage program, which is a symptom of a deeper rot in the American air-travel industry.'}]